In [2]:
%%capture
# Installs Unsloth, Xformers, and TRL
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes datasets

In [ ]:
# ==========================================
# 0. Environment Setup
# ==========================================
import os
try:
    import unsloth
except ImportError:
    print("Installing Unsloth...")
    os.system("pip install -q 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'")
    os.system("pip install -q --no-deps xformers trl peft accelerate bitsandbytes datasets")

# ==========================================
# 1. Configuration
# ==========================================
import torch
import json
import glob
import gc
import re
from unsloth import FastLanguageModel, PatchDPOTrainer
from trl import SFTTrainer, DPOTrainer, DPOConfig
from transformers import TrainingArguments
from datasets import Dataset
from unsloth.chat_templates import get_chat_template

# Paths
DATA_DIR = "/content/drive/MyDrive/ETSP"
OUTPUT_DIR_SFT = "/content/drive/MyDrive/ETSP/qwen_sft_v2"
OUTPUT_DIR_DPO = "/content/drive/MyDrive/ETSP/qwen_dpo_v2"

# Model Configuration
MODEL_NAME = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 8192  # Extended context for 30 reviews + prompt
DTYPE = None
LOAD_IN_4BIT = True

# Mount Google Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# ==========================================
# 2. Data Processing
# ==========================================
def clean_text(text: str) -> str:
    """Remove HTML tags and normalize whitespace."""
    if not isinstance(text, str):
        return ""
    if '<' in text and '>' in text:
        text = re.sub(r'<[^>]+>', '', text)
    return re.sub(r"\s+", " ", text).strip()

def format_review_context(reviews_list):
    """
    Reconstruct review context matching generation pipeline:
    - Clean text with clean_text()
    - Filter by length: 20-1500 chars
    - Format: 'Review N: {text}' (no title)
    - Max 30 reviews per product
    """
    texts = []
    count = 0

    for r in reviews_list:
        if count >= 30:
            break

        text = clean_text(r.get('text', ''))

        if len(text) < 20 or len(text) > 1500:
            continue

        texts.append(f'Review {count+1}: {text}')
        count += 1

    return '\n\n'.join(texts)

def load_datasets(data_dir):
    """
    Load *_dpo_v2.jsonl files and construct SFT/DPO datasets.

    SFT: Learns readability control (A1/C1) and format adherence
    DPO: Learns hallucination rejection via preference optimization
    """
    files = glob.glob(os.path.join(data_dir, "**/*_dpo_v2.jsonl"), recursive=True)
    print(f"Found {len(files)} data files.")

    sft_data = []
    dpo_data = []

    for file_path in files:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    item = json.loads(line)
                    reviews = item.get('reviews', [])
                    if not reviews: continue

                    context = format_review_context(reviews)
                    if not context.strip(): continue

                    simple = item.get('summary_simple')
                    complex_ = item.get('summary_complex')
                    hallucinated = item.get('summary_hallucinated')

                    if not (simple and complex_ and hallucinated): continue

                    # A1 (Simple) Prompt - matches generation exactly
                    a1_instruction = """Summarize these reviews for a beginner (CEFR A1).

Requirements:
- Use simple present tense, basic vocabulary.
- Write 1 paragraph of 3-4 short sentences.
- NO bullet points.
- Structure: [Overall] + [Feature] + [Conclusion].
- Reflect what most people say, but mention important issues if some people have them.

Reviews:
{context}

Output ONLY the summary.""".format(context=context)

                    # C1 (Complex) Prompt - matches generation exactly
                    c1_instruction = """Summarize these reviews in a professional, analytical style (CEFR C1).

Requirements:
1. **Style**: Use sophisticated vocabulary and phrasing, identical to a high-quality expert review.

2. **Format**: Use a bulleted list with EXACTLY 3-6 points total.
   - **CRITICAL**: EVERY point MUST start with `(+)`, `(-)`, or `(~)`.
   - Use `(+)` for consensus strengths.
   - Use `(-)` for consensus weaknesses.
   - Use `(~)` for mixed/controversial opinions (CRITICAL).
   - **Order**: List all `(+)` first, then `(-)`, then `(~)`. Do NOT mix them randomly.
   - **Compact output**: No blank lines between points. Each point on a new line immediately after the previous one.

3. **Handling Contradictions**:
   - If User A says "great battery" but User B says "battery died", you MUST report this as an inconsistency.
   - Use phrases like "Polarized feedback regarding...", "Inconsistent reports on...", or "While most praise X, some users note Y...".

4. **Length**: Total summary under 180 words. Each point 18-30 words.

Reviews:
{context}

Output ONLY the structured summary.""".format(context=context)

                    # SFT samples (both readability levels)
                    sft_data.append({
                        "instruction": a1_instruction,
                        "output": simple
                    })
                    sft_data.append({
                        "instruction": c1_instruction,
                        "output": complex_
                    })

                    # DPO samples (C1 only - hallucination detection)
                    dpo_data.append({
                        "question": c1_instruction,
                        "chosen": complex_,
                        "rejected": hallucinated
                    })

                except Exception:
                    continue

    print(f"Loaded: {len(sft_data)} SFT samples | {len(dpo_data)} DPO pairs")

    # 90/10 train/val split
    ds_sft = Dataset.from_list(sft_data).train_test_split(test_size=0.1, seed=42)
    ds_dpo = Dataset.from_list(dpo_data).train_test_split(test_size=0.1, seed=42)

    return ds_sft, ds_dpo

# Load datasets
dataset_sft, dataset_dpo = load_datasets(DATA_DIR)
print(f"SFT Train: {len(dataset_sft['train'])}, Val: {len(dataset_sft['test'])}")
print(f"DPO Train: {len(dataset_dpo['train'])}, Val: {len(dataset_dpo['test'])}")

# ==========================================
# 3. Stage 1: Supervised Fine-Tuning (SFT)
# ==========================================
print("\n🚀 [Stage 1] SFT Training")

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# Apply Qwen 2.5 chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
    mapping = {"role": "role", "content": "content", "user": "user", "assistant": "assistant"}
)

def format_sft_func(examples):
    """Apply chat template to SFT samples."""
    texts = []
    for inst, out in zip(examples["instruction"], examples["output"]):
        conv = [
            {"role": "user", "content": inst},
            {"role": "assistant", "content": out}
        ]
        texts.append(tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=False))
    return {"text": texts}

dataset_sft_fmt = dataset_sft.map(format_sft_func, batched=True)

# SFT Trainer
trainer_sft = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_sft_fmt['train'],
    eval_dataset = dataset_sft_fmt['test'],
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        eval_strategy = "steps",
        eval_steps = 50,
        save_strategy = "steps",
        save_steps = 100,
        save_total_limit = 3,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        optim = "adamw_8bit",
        output_dir = OUTPUT_DIR_SFT,
        seed = 3407,
    ),
)

trainer_sft.train()
print("✅ SFT complete")

model.save_pretrained(OUTPUT_DIR_SFT)
tokenizer.save_pretrained(OUTPUT_DIR_SFT)

# ==========================================
# 4. Memory Cleanup
# ==========================================
del model, tokenizer, trainer_sft
gc.collect()
torch.cuda.empty_cache()
print("🧹 Memory cleared")

# ==========================================
# 5. Stage 2: Direct Preference Optimization (DPO)
# ==========================================
print("\n🚀 [Stage 2] DPO Training")

# Reload model with SFT weights
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = OUTPUT_DIR_SFT,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

# Enable DPO training
PatchDPOTrainer()

def format_dpo_func(example):
    """Format DPO data for preference learning."""
    return {
        "prompt": tokenizer.apply_chat_template([{"role": "user", "content": example["question"]}], tokenize=False, add_generation_prompt=True),
        "chosen": example["chosen"],
        "rejected": example["rejected"],
    }

dataset_dpo_fmt = dataset_dpo.map(format_dpo_func)

# DPO Trainer
dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,  # Unsloth handles reference model internally
    tokenizer = tokenizer,
    train_dataset = dataset_dpo_fmt['train'],
    eval_dataset = dataset_dpo_fmt['test'],
    max_length = MAX_SEQ_LENGTH,
    max_prompt_length = MAX_SEQ_LENGTH - 512,  # Reserve 512 tokens for output
    beta = 0.1,  # Strong preference signal for hallucination rejection
    args = DPOConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 5e-6,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        eval_strategy = "steps",
        eval_steps = 20,
        save_strategy = "no",
        optim = "adamw_8bit",
        output_dir = OUTPUT_DIR_DPO,
        seed = 3407,
    ),
)

dpo_trainer.train()
print("✅ DPO complete")

# ==========================================
# 6. Final Export
# ==========================================
print(f"💾 Saving to {OUTPUT_DIR_DPO}")
model.save_pretrained(OUTPUT_DIR_DPO)
tokenizer.save_pretrained(OUTPUT_DIR_DPO)
print("🎉 Training complete!")

Installing Unsloth...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Found 9 data files.
Loaded: 3554 SFT samples | 1777 DPO pairs
SFT Train: 3198, Val: 356
DPO Train: 1599, Val: 178

🚀 [Stage 1] SFT Training
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Tokenizing eval dataset (num_proc=16):   0%|          | 0/178 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,599 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
20,0.852200,0.592342,13.255346,11.899546,0.711111,1.355799,-379.436462,-404.503113,-0.622687,-0.590356,0,0,0
40,0.530400,0.323342,12.162534,9.678000,0.844444,2.484532,-390.364594,-426.718628,-0.771685,-0.724624,No Log,No Log,No Log
60,0.279700,0.203208,10.942546,7.571390,0.922222,3.371155,-402.564484,-447.784668,-0.927748,-0.868945,No Log,No Log,No Log
80,0.155000,0.137324,9.493854,5.340331,0.938889,4.153523,-417.051392,-470.095276,-1.059625,-0.990322,No Log,No Log,No Log
100,0.215100,0.106404,8.069264,3.231753,0.955556,4.837512,-431.297302,-491.181122,-1.188316,-1.110057,No Log,No Log,No Log
120,0.175700,0.089143,6.999538,1.613979,0.961111,5.385560,-441.994537,-507.358826,-1.277033,-1.191818,No Log,No Log,No Log
140,0.129700,0.078113,6.081824,0.248831,0.966667,5.832993,-451.171661,-521.010254,-1.358885,-1.266958,No Log,No Log,No Log
160,0.037800,0.072495,5.693827,-0.391511,0.966667,6.085338,-455.051697,-527.413696,-1.390653,-1.296496,No Log,No Log,No Log
180,0.068900,0.070677,5.400839,-0.835406,0.966667,6.236245,-457.981567,-531.852661,-1.414287,-1.317963,No Log,No Log,No Log
200,0.120300,0.069946,5.352250,-0.913489,0.966667,6.265739,-458.467438,-532.633484,-1.418352,-1.321566,No Log,No Log,No Log


eval/logits/chosen,█▇▅▄▃▂▂▁▁▁
eval/logits/rejected,█▇▅▄▃▂▂▁▁▁
eval/logps/chosen,█▇▆▅▃▂▂▁▁▁
eval/logps/rejected,█▇▆▄▃▂▂▁▁▁
eval/loss,█▄▃▂▁▁▁▁▁▁
eval/rewards/accuracies,▁▅▇▇██████
eval/rewards/chosen,█▇▆▅▃▂▂▁▁▁
eval/rewards/margins,▁▃▄▅▆▇▇███
eval/rewards/rejected,█▇▆▄▃▂▂▁▁▁
eval/runtime,▆▁▃▃▂▂▂▃█▅
+15,...


✅ DPO complete
💾 Saving to /content/drive/MyDrive/ETSP/qwen_dpo_v2
🎉 Training complete!


In [3]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
import json
import re
import random
import os
import glob

class Colors:
    GREEN = '\033[92m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    CYAN = '\033[96m'
    BOLD = '\033[1m'
    RESET = '\033[0m'
    GRAY = '\033[90m'

MODEL_PATH = "/content/drive/MyDrive/ETSP/qwen_dpo_v2"
DATA_DIR = "/content/drive/MyDrive/ETSP"

print(f"{Colors.CYAN}正在加载训练好的 DPO 模型: {MODEL_PATH}...{Colors.RESET}")
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = MODEL_PATH,
        max_seq_length = 6144,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)
    print(f"{Colors.GREEN}模型加载成功!{Colors.RESET}\n")
except Exception as e:
    print(f"{Colors.RED}模型加载失败: {e}{Colors.RESET}")
    exit()

def clean_text(text: str) -> str:
    if not isinstance(text, str): return ""
    if '<' in text and '>' in text:
        text = re.sub(r'<[^>]+>', '', text)
    return re.sub(r"\s+", " ", text).strip()

def format_review_context(reviews_list):
    texts = []
    count = 0
    for r in reviews_list:
        if count >= 30: break
        text = clean_text(r.get('text', ''))
        if len(text) < 20 or len(text) > 1500: continue
        texts.append(f'Review {count+1}: {text}')
        count += 1
    return '\n\n'.join(texts)

def load_random_sample():
    files = glob.glob(os.path.join(DATA_DIR, "**/*_dpo_v2.jsonl"), recursive=True)
    if not files:
        print(f"{Colors.RED}找不到测试数据文件 (*_dpo_v2.jsonl){Colors.RESET}")
        return None, None

    target_file = random.choice(files)
    print(f"使用数据文件: {target_file}")

    samples = []
    with open(target_file, 'r', encoding='utf-8') as f:
        for _ in range(500):
            line = f.readline()
            if not line: break
            samples.append(json.loads(line))

    return random.choice(samples), target_file

sample_data, filename = load_random_sample()
if not sample_data: exit()

context_input = format_review_context(sample_data['reviews'])
asin = sample_data.get('parent_asin', 'Unknown')
category = sample_data.get('category', 'Unknown')

print(f"{'='*60}")
print(f"产品: {category} | ASIN: {asin}")
print(f"原始评论输入 (共 {len(sample_data['reviews'])} 条, 截取前30条)")
print(f"{'='*60}")
print(f"{Colors.GRAY}{context_input[:1000]}... [由于太长只显示前1000字符]{Colors.RESET}")
print(f"{'='*60}\n")

def get_prompt(style, context):
    if style == "A1":
        return """Summarize these reviews for a beginner (CEFR A1).

Requirements:
- Use simple present tense, basic vocabulary.
- Write 1 paragraph of 3-4 short sentences.
- NO bullet points.
- Structure: [Overall] + [Feature] + [Conclusion].
- Reflect what most people say, but mention important issues if some people have them.

Reviews:
{context}

Output ONLY the summary.""".format(context=context)

    elif style == "C1":
        return """Summarize these reviews in a professional, analytical style (CEFR C1).

Requirements:
1. **Style**: Use sophisticated vocabulary and phrasing, identical to a high-quality expert review.

2. **Format**: Use a bulleted list with EXACTLY 3-6 points total.
   - **CRITICAL**: EVERY point MUST start with `(+)`, `(-)`, or `(~)`.
   - Use `(+)` for consensus strengths.
   - Use `(-)` for consensus weaknesses.
   - Use `(~)` for mixed/controversial opinions (CRITICAL).
   - **Order**: List all `(+)` first, then `(-)`, then `(~)`. Do NOT mix them randomly.
   - **Compact output**: No blank lines between points. Each point on a new line immediately after the previous one.

3. **Handling Contradictions**:
   - If User A says "great battery" but User B says "battery died", you MUST report this as an inconsistency.
   - Use phrases like "Polarized feedback regarding...", "Inconsistent reports on...", or "While most praise X, some users note Y...".

4. **Length**: Total summary under 180 words. Each point 18-30 words.

Reviews:
{context}

Output ONLY the structured summary.""".format(context=context)

def run_test(style):
    print(f"\n>>> 测试模式: {Colors.BOLD}{style} (SFT+DPO 效果验证){Colors.RESET}")

    user_prompt = get_prompt(style, context_input)

    messages = [{"role": "user", "content": user_prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    print(f"{Colors.CYAN}模型生成中...{Colors.RESET}")
    print(f"{Colors.YELLOW}{'-'*40}{Colors.RESET}")

    _ = model.generate(
        inputs,
        streamer=streamer,
        max_new_tokens=512,
        temperature=0.1,
        use_cache=True
    )
    print(f"{Colors.YELLOW}{'-'*40}{Colors.RESET}")

run_test("A1")
run_test("C1")
print(f"\n{Colors.GREEN}测试完成{Colors.RESET}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
正在加载训练好的 DPO 模型: /content/drive/MyDrive/ETSP/qwen_dpo_v2...
==((====))==  Unsloth 2025.12.7: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Unsloth 2025.12.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


模型加载成功!

使用数据文件: /content/drive/MyDrive/ETSP/Pet_Supplies/Pet_Supplies_dpo_v2.jsonl


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


产品: Pet_Supplies | ASIN: B01MD0MV9M
原始评论输入 (共 5 条, 截取前30条)
Review 1: We have been a fan of this carrier for years. It is comfortable to carry, the cats love them. They are sturdy and a bonus is that they look great.Only down side is that they are not the easiest to clean. We have put them through the washer a couple of times (the big one at the laundry mat) when a cat had an accident, even though you are not supposed to and they survived just fine but I would not do so regularly.They work great on planes (used on the flight back from Germany with our 15# cat) and for trips to the vet. It is actually possible AND comfortable to carry and stack multiple carriers!!They are expensive so we have slowly relpaced our cheaper ones and now everyone has a sleepy pod so they can stop fighting over them. Heavier cats will fit in this carrier since they can turn around, but I would not make them travel for long in it (just to the vet in our case, the 19# cat gets a BIG hard sided carrier if he flys

In [ ]:
import os
import json
import random
import re
from tqdm.auto import tqdm
from unsloth import FastLanguageModel
from datasets import Dataset
from openai import OpenAI

MODEL_PATH = "/content/drive/MyDrive/ETSP/qwen_dpo_v2"
DATA_ROOT = "/content/drive/MyDrive/ETSP"

CATEGORIES = [
    "Electronics", "Books", "Home_and_Kitchen", "Beauty_and_Personal_Care",
    "Clothing_Shoes_and_Jewelry", "Toys_and_Games", "Sports_and_Outdoors",
    "Pet_Supplies", "Automotive", "Office_Products"
]

SAMPLES_PER_CATEGORY = 5

API_KEY = "sk-xxxxxxxxxxxxxxxxxxx"
BASE_URL = "https://dashscope.aliyuncs.com/compatible-mode/v1"
JUDGE_MODEL = "qwen-plus"

print(f"正在加载微调模型: {MODEL_PATH}...")
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = MODEL_PATH,
        max_seq_length = 6144,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)
    print("Student 模型加载成功")
except Exception as e:
    print(f"模型加载失败: {e}")
    exit()

def clean_text(text: str) -> str:
    if not isinstance(text, str): return ""
    if '<' in text and '>' in text: text = re.sub(r'<[^>]+>', '', text)
    return re.sub(r"\s+", " ", text).strip()

def format_context(reviews):
    texts = []
    count = 0
    for r in reviews:
        if count >= 30: break
        text = clean_text(r.get('text', ''))
        if len(text) < 20 or len(text) > 1500: continue
        texts.append(f'Review {count+1}: {text}')
        count += 1
    return '\n\n'.join(texts)

def get_unseen_samples(category, count=2):
    file_path = os.path.join(DATA_ROOT, category, f"{category}_dpo_v2.jsonl")
    if not os.path.exists(file_path):
        file_path = os.path.join(DATA_ROOT, category, f"{category}_dpo.jsonl")
        if not os.path.exists(file_path):
            print(f"跳过 {category}: 文件不存在")
            return []

    clean_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line)
                safe_item = {
                    "parent_asin": item.get("parent_asin", "unknown"),
                    "reviews": item.get("reviews", []),
                }
                clean_data.append(safe_item)
            except:
                continue

    if len(clean_data) < 10: return []

    try:
        ds = Dataset.from_list(clean_data)
        ds_split = ds.train_test_split(test_size=0.1, seed=42)
        test_set = ds_split['test']

        if len(test_set) > count:
            indices = random.sample(range(len(test_set)), count)
            samples = [test_set[i] for i in indices]
        else:
            samples = [item for item in test_set]

        results = []
        for item in samples:
            context = format_context(item.get('reviews', []))
            if len(context) > 200:
                item['formatted_context'] = context
                item['category'] = category
                results.append(item)

        return results

    except Exception as e:
        print(f"{category} 处理出错: {e}")
        return []

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

def evaluate_factuality(original_reviews, generated_summary, style):
    if style == "A1":
        style_requirements = """
[A1 格式要求]
- 应该是 1 段话，3-4 个简单短句
- 使用简单现在时和基础词汇
- 不应该有 bullet points
- 结构：[总体评价] + [特点] + [结论]
"""
    else:  # C1
        style_requirements = """
[C1 格式要求]
- 应该是 3-6 个 bullet points
- 每个 point 必须以 (+) / (-) / (~) 开头
- 顺序：所有 (+) 在前，然后 (-)，最后 (~)
- 使用复杂词汇和专业表达
"""

    judge_prompt = f"""
你是一位公平、客观的事实核查员。请基于下方的评论原文，检查 AI 生成的摘要内容是否属实。

[输入数据]
评论原文 (Source):
{original_reviews}

AI 生成摘要 (Summary):
{generated_summary}

{style_requirements}

[判罚标准]

PASS (合格):
1. **有据可依**: 摘要中提到的观点（无论是优点还是缺点），只要在评论原文中真实出现过（哪怕只有少数人提到），就算正确。
2. **合理归纳**: 允许模型对相似的观点进行总结概括。
3. **格式正确**: 符合 {style} 的格式要求。

FAIL (不合格):
1. **无中生有 (Fabrication)**: 提到了评论中根本不存在的参数、功能、配件或具体场景。
2. **情感反转 (Sentiment Error)**: 评论明明说是"缺点"，摘要却写成了"优点"（或反之）。
3. **掩盖矛盾 (Misrepresentation)**: 如果评论中对某一点存在明显分歧（有人说好，有人说坏），但摘要却只单方面说"大家都喜欢"，忽略了反面声音。
4. **格式错误**: 不符合 {style} 的格式要求。

[输出格式 - JSON]
{{
    "score": <0-10>,
    "is_factual": <boolean>,
    "format_correct": <boolean>,
    "hallucinations": [
        "摘要原句 -> 错误原因"
    ],
    "reasoning": "简短评价。"
}}
"""
    try:
        response = client.chat.completions.create(
            model=JUDGE_MODEL,
            messages=[{"role": "user", "content": judge_prompt}],
            temperature=0.0,
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Judge API Error: {e}")
        return {"score": 0, "is_factual": False, "format_correct": False, "hallucinations": ["API Error"], "reasoning": str(e)}

def generate_student_summary(context, style="C1"):
    if style == "A1":
        user_prompt = """Summarize these reviews for a beginner (CEFR A1).

Requirements:
- Use simple present tense, basic vocabulary.
- Write 1 paragraph of 3-4 short sentences.
- NO bullet points.
- Structure: [Overall] + [Feature] + [Conclusion].
- Reflect what most people say, but mention important issues if some people have them.

Reviews:
{context}

Output ONLY the summary.""".format(context=context)

    else:  # C1
        user_prompt = """Summarize these reviews in a professional, analytical style (CEFR C1).

Requirements:
1. **Style**: Use sophisticated vocabulary and phrasing.
2. **Format**: Use a bulleted list with EXACTLY 3-6 points total.
   - **CRITICAL**: EVERY point MUST start with `(+)`, `(-)`, or `(~)`.
   - Use `(+)` for consensus strengths.
   - Use `(-)` for consensus weaknesses.
   - Use `(~)` for mixed/controversial opinions.
   - **Order**: List all `(+)` first, then `(-)`, then `(~)`.

3. **Handling Contradictions**:
   - Report inconsistencies using phrases like "Polarized feedback...".

4. **Length**: Total summary under 180 words.

Reviews:
{context}

Output ONLY the structured summary.""".format(context=context)

    messages = [{"role": "user", "content": user_prompt}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        temperature=0.1,
        use_cache=True
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "assistant" in text:
        return text.split("assistant")[-1].strip()
    return text

def main():
    all_test_samples = []
    print(f"\n正在从 {len(CATEGORIES)} 个品类中抽取未见验证集 (seed=42)...")

    for cat in tqdm(CATEGORIES, desc="Sampling"):
        samples = get_unseen_samples(cat, count=SAMPLES_PER_CATEGORY)
        all_test_samples.extend(samples)

    if not all_test_samples:
        print("未找到任何测试样本，请检查数据路径")
        return

    print(f"\n准备就绪，共 {len(all_test_samples)} 个样本。开始全品类真实性测试\n")

    results_a1 = []
    results_c1 = []

    GREEN = '\033[92m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    CYAN = '\033[96m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

    for i, item in enumerate(all_test_samples):
        cat = item['category']
        asin = item['parent_asin']
        context = item['formatted_context']

        print(f"\n{'='*60}")
        print(f"[{i+1}/{len(all_test_samples)}] 测试产品: {cat} (ASIN: {asin})")
        print(f"{'='*60}")

        # 测试 A1 模式
        print(f"\n{CYAN}{BOLD}>>> A1 模式测试{RESET}")
        summary_a1 = generate_student_summary(context, style="A1")
        eval_a1 = evaluate_factuality(context, summary_a1, style="A1")

        res_a1 = {
            "category": cat,
            "asin": asin,
            "style": "A1",
            "summary": summary_a1,
            "score": eval_a1['score'],
            "is_factual": eval_a1['is_factual'],
            "format_correct": eval_a1.get('format_correct', True),
            "hallucinations": eval_a1['hallucinations'],
            "reasoning": eval_a1.get('reasoning', '')
        }
        results_a1.append(res_a1)

        score_a1 = eval_a1['score']
        color_a1 = GREEN if score_a1 >= 8 else (YELLOW if score_a1 >= 5 else RED)

        print(f"   生成摘要: {summary_a1[:100].replace(chr(10), ' ')}...")
        print(f"   裁判打分: {color_a1}{score_a1}/10{RESET}")
        if eval_a1['hallucinations']:
            print(f"   幻觉/错误: {RED}{eval_a1['hallucinations']}{RESET}")
        else:
            print(f"   事实核查通过")

        # 测试 C1 模式
        print(f"\n{CYAN}{BOLD}>>> C1 模式测试{RESET}")
        summary_c1 = generate_student_summary(context, style="C1")
        eval_c1 = evaluate_factuality(context, summary_c1, style="C1")

        res_c1 = {
            "category": cat,
            "asin": asin,
            "style": "C1",
            "summary": summary_c1,
            "score": eval_c1['score'],
            "is_factual": eval_c1['is_factual'],
            "format_correct": eval_c1.get('format_correct', True),
            "hallucinations": eval_c1['hallucinations'],
            "reasoning": eval_c1.get('reasoning', '')
        }
        results_c1.append(res_c1)

        score_c1 = eval_c1['score']
        color_c1 = GREEN if score_c1 >= 8 else (YELLOW if score_c1 >= 5 else RED)

        print(f"   生成摘要: {summary_c1[:100].replace(chr(10), ' ')}...")
        print(f"   裁判打分: {color_c1}{score_c1}/10{RESET}")
        if eval_c1['hallucinations']:
            print(f"   幻觉/错误: {RED}{eval_c1['hallucinations']}{RESET}")
        else:
            print(f"   事实核查通过")

        print("-" * 60)

    if not results_a1 and not results_c1: return

    # 最终统计报告
    print("\n" + "="*70)
    print("幻觉检测最终报告 (Final Factuality Report)")
    print("="*70)

    def print_stats(results, style_name):
        print(f"\n{BOLD}【{style_name} 模式统计】{RESET}")

        cat_stats = {}
        for r in results:
            c = r['category']
            if c not in cat_stats: cat_stats[c] = []
            cat_stats[c].append(r['score'])

        print(f"{'Category':<30} | {'Avg Score':<10}")
        print("-" * 45)
        for cat, scores in cat_stats.items():
            avg = sum(scores)/len(scores)
            print(f"{cat:<30} | {avg:.1f}")

        total_avg = sum(r['score'] for r in results) / len(results)
        pass_count = sum(1 for r in results if r['is_factual'])
        format_count = sum(1 for r in results if r.get('format_correct', True))
        pass_rate = (pass_count / len(results)) * 100
        format_rate = (format_count / len(results)) * 100

        print("-" * 45)
        print(f"总平均分: {total_avg:.2f} / 10")
        print(f"无幻觉通过率: {pass_rate:.1f}%")
        print(f"格式正确率: {format_rate:.1f}%")

    print_stats(results_a1, "A1")
    print_stats(results_c1, "C1")

    print("\n" + "="*70)

    # 保存结果
    all_results = {
        "A1": results_a1,
        "C1": results_c1
    }

    with open("hallucination_test_results_full.json", "w", encoding='utf-8') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print("详细结果已保存至 hallucination_test_results_full.json")

if __name__ == "__main__":
    main()


正在加载微调模型: /content/drive/MyDrive/ETSP/qwen_dpo_v2...
==((====))==  Unsloth 2025.12.7: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Student 模型加载成功

正在从 10 个品类中抽取未见验证集 (seed=42)...


Sampling:   0%|          | 0/10 [00:00<?, ?it/s]


准备就绪，共 50 个样本。开始全品类真实性测试


[1/50] 测试产品: Electronics (ASIN: B089Q5MJ2K)

>>> A1 模式测试
   生成摘要: Most people like the laptop because it is fast, thin, and good for work or hobbies. Many say it work...
   裁判打分: 10/10
   事实核查通过

>>> C1 模式测试
   生成摘要: (+) Exceptional performance, rapid startup, lightweight design, strong value for money, and high rel...
   裁判打分: 9/10
   事实核查通过
------------------------------------------------------------

[2/50] 测试产品: Electronics (ASIN: B0B2CX4CKD)

>>> A1 模式测试
   生成摘要: Most people like this microphone because it has good sound, easy setup, and cool features like light...
   裁判打分: 10/10
   事实核查通过

>>> C1 模式测试
   生成摘要: **Strengths:** - **Sound Quality:** Consistently praised for clear, crisp audio, particularly in gam...
   裁判打分: 9/10
   事实核查通过
------------------------------------------------------------

[3/50] 测试产品: Electronics (ASIN: B084GF686M)

>>> A1 模式测试
   生成摘要: Most people like these drives because they are cheap and work well for simple tasks. Some ha